In [32]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import pad_sequences

In [33]:
# --- CONFIGURAÇÕES ---
hp = {
    'MAX_WORD_INDEX': 1000,
    'MAX_LEN': 300,       # Sequências inicial (30)
    'N_STATES': 3 ,       # N
    'N_ITER': 30          # Iterações iniciais (5)
}

print("1. Carregando dados...")
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=hp['MAX_WORD_INDEX'])

# Separamos os dados por classe para treinar os HMM
# Modelo Positivo só vê reviews positivos (label 1)
# Modelo Negativo só vê reviews negativos (label 0)
train_pos = [x for x, y in zip(train_data, train_labels) if y == 1]
train_neg = [x for x, y in zip(train_data, train_labels) if y == 0]

# Quantidade para o treino
LIMIT_TRAIN = 300
train_pos = pad_sequences(train_pos[:LIMIT_TRAIN], maxlen=hp['MAX_LEN'])
train_neg = pad_sequences(train_neg[:LIMIT_TRAIN], maxlen=hp['MAX_LEN'])
X_test = pad_sequences(test_data[:100], maxlen=hp['MAX_LEN']) # Teste pequeno
y_test_sample = test_labels[:100]

# --- CLASSE HMM ---

class HMMManual:
    def __init__(self, n_states, n_emissions):
        self.N = n_states
        self.M = n_emissions

        # Inicialização aleatória e normalizada
        np.random.seed(42)
        self.pi = np.random.rand(self.N)
        self.pi /= np.sum(self.pi)

        self.A = np.random.rand(self.N, self.N)
        self.A /= self.A.sum(axis=1, keepdims=True)

        self.B = np.random.rand(self.N, self.M)
        self.B /= self.B.sum(axis=1, keepdims=True)

    def _forward(self, obs):
        """Calcula alpha (probabilidade de chegar no estado i no tempo t)"""
        T = len(obs)
        alpha = np.zeros((T, self.N))
        scale = np.zeros(T) # Fator de escala para evitar underflow

        # Inicialização
        alpha[0] = self.pi * self.B[:, obs[0]]
        scale[0] = np.sum(alpha[0]) + 1e-10 # +1e-10 evita divisão por zero
        alpha[0] /= scale[0]

        # Indução (indo)
        for t in range(1, T):
            for j in range(self.N):
                alpha[t, j] = np.sum(alpha[t-1] * self.A[:, j]) * self.B[j, obs[t]]

            scale[t] = np.sum(alpha[t]) + 1e-10
            alpha[t] /= scale[t]

        return alpha, scale

    def _backward(self, obs, scale):
        """Calcula beta (probabilidade do futuro dado o estado i no tempo t)"""
        T = len(obs)
        beta = np.zeros((T, self.N))

        # Inicialização (no tempo T, beta = 1)
        beta[T-1] = 1.0 / scale[T-1] # Usa a mesma escala do forward

        # Indução Reversa (vindo)
        for t in range(T-2, -1, -1):
            for i in range(self.N):
                # transição * emissão_futura * beta_futuro
                beta[t, i] = np.sum(self.A[i, :] * self.B[:, obs[t+1]] * beta[t+1, :])

            beta[t] /= scale[t] # Normaliza com escala do forward

        return beta

    def train(self, sequences, n_iter=5):
        for it in range(n_iter):
            # A_num[i, j] = Esperança de transições de i para j
            A_num = np.zeros((self.N, self.N))
            A_den = np.zeros((self.N, 1))

            # B_num[j, k] = Esperança de estar em j e emitir k
            B_num = np.zeros((self.N, self.M))
            B_den = np.zeros((self.N, 1))

            Pi_new = np.zeros(self.N)

            # E-STEP: Calcula estatísticas para cada sequência
            for seq in sequences:
                # Remove padding (0) para o treino
                obs = [x for x in seq if x != 0]
                if len(obs) < 2: continue

                T = len(obs)
                alpha, scale = self._forward(obs)
                beta = self._backward(obs, scale)

                # Calcula Gamma (Prob de estar no estado i no tempo t)
                # gamma[t, i] ~ alpha * beta
                gamma = alpha * beta
                # Normaliza gamma
                gamma /= (np.sum(gamma, axis=1, keepdims=True) + 1e-10)

                # Calcula Xi (Prob de transição i -> j no tempo t)
                xi = np.zeros((T-1, self.N, self.N))
                for t in range(T-1):
                    denom = np.sum(alpha[t,:] * np.sum(self.A * self.B[:, obs[t+1]] * beta[t+1, :], axis=1)) + 1e-10
                    for i in range(self.N):
                        xi[t, i, :] = alpha[t, i] * self.A[i, :] * self.B[:, obs[t+1]] * beta[t+1, :] / denom

                # M-STEP (Acumulação): Soma as ocorrências esperadas
                Pi_new += gamma[0]

                # Atualiza acumuladores de A
                A_num += np.sum(xi, axis=0)
                A_den += np.sum(gamma[:-1], axis=0).reshape(-1, 1)

                # Atualiza acumuladores de B
                for t in range(T):
                    symbol = obs[t]
                    B_num[:, symbol] += gamma[t]
                B_den += np.sum(gamma, axis=0).reshape(-1, 1)

            # ATUALIZAÇÃO FINAL DOS PARÂMETROS
            self.pi = Pi_new / len(sequences)
            self.A = A_num / (A_den + 1e-10) # +1e-10 evita div por 0
            self.B = B_num / (B_den + 1e-10)

            print(f"  > Iteração {it+1}/{n_iter} concluída.")

    def score(self, observation_sequence):
        """Retorna Log-Likelihood (usado para classificar)"""
        obs = [o for o in observation_sequence if o != 0]
        if len(obs) == 0: return -np.inf

        _, scale = self._forward(obs)
        # Log da probabilidade é a soma dos logs das escalas
        return np.sum(np.log(scale + 1e-10))

# --- TRAINING LOOP ---

print("\n2. Treinando Modelo POSITIVO (Isso pode demorar um pouco)...")
hmm_pos = HMMManual(hp['N_STATES'], hp['MAX_WORD_INDEX'] + 1)
hmm_pos.train(train_pos, n_iter=hp['N_ITER'])

print("\n3. Treinando Modelo NEGATIVO...")
hmm_neg = HMMManual(hp['N_STATES'], hp['MAX_WORD_INDEX'] + 1)
hmm_neg.train(train_neg, n_iter=hp['N_ITER'])

print("\n4. Testando Acurácia...")
acertos = 0
for i in range(len(X_test)):
    seq = X_test[i]
    if len([x for x in seq if x != 0]) < 2: continue

    # Classificação Bayesiana (De quem é mais provável ser essa frase)
    score_pos = hmm_pos.score(seq)
    score_neg = hmm_neg.score(seq)

    pred = 1.0 if score_pos > score_neg else 0.0

    if pred == y_test_sample[i]:
        acertos += 1

print(f"\n=== RESULTADO FINAL ===")
print(f"Acurácia no teste ({len(X_test)} amostras): {acertos/len(X_test):.2f}")

1. Carregando dados...

2. Treinando Modelo POSITIVO (Isso pode demorar um pouco)...
  > Iteração 1/30 concluída.
  > Iteração 2/30 concluída.
  > Iteração 3/30 concluída.
  > Iteração 4/30 concluída.
  > Iteração 5/30 concluída.
  > Iteração 6/30 concluída.
  > Iteração 7/30 concluída.
  > Iteração 8/30 concluída.
  > Iteração 9/30 concluída.
  > Iteração 10/30 concluída.
  > Iteração 11/30 concluída.
  > Iteração 12/30 concluída.
  > Iteração 13/30 concluída.
  > Iteração 14/30 concluída.
  > Iteração 15/30 concluída.
  > Iteração 16/30 concluída.
  > Iteração 17/30 concluída.
  > Iteração 18/30 concluída.
  > Iteração 19/30 concluída.
  > Iteração 20/30 concluída.
  > Iteração 21/30 concluída.
  > Iteração 22/30 concluída.
  > Iteração 23/30 concluída.
  > Iteração 24/30 concluída.
  > Iteração 25/30 concluída.
  > Iteração 26/30 concluída.
  > Iteração 27/30 concluída.
  > Iteração 28/30 concluída.
  > Iteração 29/30 concluída.
  > Iteração 30/30 concluída.

3. Treinando Modelo NEG

In [37]:
# --- Teste ---
import numpy as np
from tensorflow.keras.datasets import imdb

# Garantir que usamos o mesmo mapeamento que o Keras usou no treino
word_index = imdb.get_word_index()
# O Keras reserva os primeiros índices: 0=Padding, 1=Start, 2=Unknown
word_index = {k: (v + 3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2

def classificar_frase(texto):
    # Limpeza
    texto_limpo = texto.lower().replace('.', '').replace(',', '').replace('!', '').replace('?', '')
    palavras = texto_limpo.split()

    # Tradução
    seq_numerica = []
    for w in palavras:
        if w in word_index:
            idx = word_index[w]
            if idx >= hp['MAX_WORD_INDEX']:
                seq_numerica.append(2) # 2 = Desconhecido
            else:
                seq_numerica.append(idx)
        else:
            seq_numerica.append(2) # Palavra que não existe no IMDB vira <UNK>

    # Formatação para o HMM
    seq_formatada = np.array([x for x in seq_numerica if x != 0])

    if len(seq_formatada) == 0:
        return "Frase vazia ou palavras fora do vocabulário conhecido."

    # Bayes (Score nos dois modelos)
    log_prob_pos = hmm_pos.score(seq_formatada) # Modelo Positivo
    log_prob_neg = hmm_neg.score(seq_formatada) # Modelo Negativo

    # Decisão
    print(f"Frase: \"{texto}\"")
    print(f"Score Positivo: {log_prob_pos:.2f}")
    print(f"Score Negativo: {log_prob_neg:.2f}")

    if log_prob_pos > log_prob_neg:
        return ">> RESULTADO: 😄 POSITIVO"
    else:
        return ">> RESULTADO: 😡 NEGATIVO"



frase_teste = "I hated this movie it is the worst film ever terrible story"

# Executa a classificação
print(classificar_frase(frase_teste))

Frase: "I hated this movie it is the worst film ever terrible story"
Score Positivo: -66.62
Score Negativo: -62.21
>> RESULTADO: 😡 NEGATIVO
